## Assignment : 14

<pre>
1. Preprocess all the Data we have in DonorsChoose <a href='https://drive.google.com/drive/folders/1MIwK7BQMev8f5CbDDVNLPaFGB32pFN60'>Dataset</a> use train.csv
2. Combine 4 essay's into one column named - 'preprocessed_essays'. 
3. After step 2 you have to train 3 types of models as discussed below. 
4. For all the model use <a href='https://scikit-learn.org/stable/modules/model_evaluation.html#roc-metrics'>'auc'</a> as a metric. check <a href='https://datascience.stackexchange.com/a/20192'>this</a> for using auc as a metric 
5. You are free to choose any number of layers/hiddden units but you have to use same type of architectures shown below. 
6. You can use any one of the optimizers and choice of Learning rate and momentum, resources: <a href='http://cs231n.github.io/neural-networks-3/'>cs231n class notes</a>, <a href='https://www.youtube.com/watch?v=hd_KFJ5ktUc'>cs231n class video</a>. 
7. For all the model's use <a href='https://www.youtube.com/watch?v=2U6Jl7oqRkM'>TensorBoard</a> and plot the Metric value and Loss with epoch. While submitting, take a screenshot of plots and include those images in .ipynb notebook and PDF. 
8. Use Categorical Cross Entropy as Loss to minimize.
</pre>

### Model-1

Build and Train deep neural network as shown below

<img src='https://i.imgur.com/w395Yk9.png'>
ref: https://i.imgur.com/w395Yk9.png

- __Input_seq_total_text_data__ --- You have to give Total text data columns. After this use the Embedding layer to get word vectors. Use given predefined glove word vectors, don't train any word vectors. After this use LSTM and get the LSTM output and Flatten that output. 
- __Input_school_state__ --- Give 'school_state' column as input to embedding layer and Train the Keras Embedding layer. 
- __Project_grade_category__  --- Give 'project_grade_category' column as input to embedding layer and Train the Keras Embedding layer.
- __Input_clean_categories__ --- Give 'input_clean_categories' column as input to embedding layer and Train the Keras Embedding layer.
- __Input_clean_subcategories__ --- Give 'input_clean_subcategories' column as input to embedding layer and Train the Keras Embedding layer.
- __Input_clean_subcategories__ --- Give 'input_teacher_prefix' column as input to embedding layer and Train the Keras Embedding layer.
- __Input_remaining_teacher_number_of_previously_posted_projects._resource_summary_contains_numerical_digits._price._quantity__ ---concatenate remaining columns and add a Dense layer after that. 



- For LSTM, you can choose your sequence padding methods on your own or you can train your LSTM without padding, there is no restriction on that.

Below is an example of embedding layer for a categorical columns. In below code all are dummy values, we gave only for referance. 

In [0]:
# https://stats.stackexchange.com/questions/270546/how-does-keras-embedding-layer-work
input_layer = Input(shape=(n,))
embedding = Embedding(no_1, no_2, input_length=n)(input_layer)
flatten = Flatten()(embedding)

In [1]:
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

import sqlite3
import pandas as pd
import numpy as np
import nltk
import string
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
from nltk.stem.porter import PorterStemmer
from numpy import zeros
from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Input
from keras.layers import Embedding
from keras.layers import LSTM,Bidirectional
from keras.layers.core import Dense, Dropout
from keras.models import Model, load_model
from keras.layers.normalization import BatchNormalization
from keras.callbacks import ReduceLROnPlateau
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical

import keras
from tensorboardcolab import *
from keras.regularizers import l2
from keras.layers import LeakyReLU


Using TensorFlow backend.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
df = pd.read_csv('/content/drive/My Drive/Applied ML assignments/preprocessed_data.csv')

In [4]:
resource_data = pd.read_csv('/content/drive/My Drive/LSTM Assignment/resources.csv')
resource_data.columns
project_data = pd.read_csv('/content/drive/My Drive/LSTM Assignment/train_data.csv')
project_data.columns

Index(['Unnamed: 0', 'id', 'teacher_id', 'teacher_prefix', 'school_state',
       'project_submitted_datetime', 'project_grade_category',
       'project_subject_categories', 'project_subject_subcategories',
       'project_title', 'project_essay_1', 'project_essay_2',
       'project_essay_3', 'project_essay_4', 'project_resource_summary',
       'teacher_number_of_previously_posted_projects', 'project_is_approved'],
      dtype='object')

In [5]:
price_data = resource_data.groupby('id').agg({'price':'sum', 'quantity':'sum'}).reset_index()
price_data.head(2)

,id,price,quantity
0,p000001,459.56,7
1,p000002,515.89,21


In [0]:
project_data = pd.merge(project_data, price_data, on='id', how='left')

In [7]:
project_data['quantity'].shape

(109248,)

In [0]:
df['quantity'] = project_data['quantity']
#df1['columename']= df2['existing_colume_name']

In [9]:
df.columns

Index(['school_state', 'teacher_prefix', 'project_grade_category',
       'teacher_number_of_previously_posted_projects', 'project_is_approved',
       'clean_categories', 'clean_subcategories', 'essay', 'price',
       'quantity'],
      dtype='object')

In [0]:
#assigning class labels
y=df['project_is_approved']
df.drop(['project_is_approved'],axis=1, inplace=True)
x=df

In [85]:
#x.columns
#x.drop(['teacher_number_of_previously_posted_projects','price','quantity'],axis=1, inplace=True)
x.columns
#col = ['teacher_prefix', 'school_state', 'project_grade_category',
       #'clean_categories', 'clean_subcategories','essay',
       #'remaining_input']
#x = x[col]
#x.columns

Index(['school_state', 'teacher_prefix', 'project_grade_category',
       'teacher_number_of_previously_posted_projects', 'clean_categories',
       'clean_subcategories', 'essay', 'price', 'quantity'],
      dtype='object')

In [11]:
#Splitting into train and test data
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)
#Splitting train data into train and cv(60:20)
X_tr, X_cv, y_tr, y_cv = train_test_split(X_train, y_train, test_size=0.2)
print(X_tr.shape, y_tr.shape)
print(X_cv.shape, y_cv.shape)

(87398, 9) (87398,)
(21850, 9) (21850,)
(69918, 9) (69918,)
(17480, 9) (17480,)


In [0]:
#https://stackoverflow.com/questions/21057621/sklearn-labelencoder-with-never-seen-before-values
from sklearn.preprocessing import LabelEncoder
class LabelEncoderExt(object):
    def __init__(self):
        """
        It differs from LabelEncoder by handling new classes and providing a value for it [Unknown]
        Unknown will be added in fit and transform will take care of new item. It gives unknown class id
        """
        self.label_encoder = LabelEncoder()
        # self.classes_ = self.label_encoder.classes_

    def fit(self, data_list):
        """
        This will fit the encoder for all the unique values and introduce unknown value
        :param data_list: A list of string
        :return: self
        """
        self.label_encoder = self.label_encoder.fit(list(data_list) + ['Unknown'])
        self.classes_ = self.label_encoder.classes_

        return self

    def transform(self, data_list):
        """
        This will transform the data_list to id list where the new values get assigned to Unknown class
        :param data_list:
        :return:
        """
        new_data_list = list(data_list)
        for unique_item in np.unique(data_list):
            if unique_item not in self.label_encoder.classes_:
                new_data_list = ['Unknown' if x==unique_item else x for x in new_data_list]

        return self.label_encoder.transform(new_data_list)

In [13]:
#teacher_prefix
vectorizer = LabelEncoderExt()
vectorizer.fit(X_tr['teacher_prefix'].values)
teacherprefix_ohe_train = vectorizer.transform(X_tr['teacher_prefix'].values)
teacherprefix_ohe_cv = vectorizer.transform(X_cv['teacher_prefix'].values)
teacherprefix_ohe_test = vectorizer.transform(X_test['teacher_prefix'].values)
print(teacherprefix_ohe_cv.shape)
print(teacherprefix_ohe_train.shape)
print(teacherprefix_ohe_test.shape)

(17480,)
(69918,)
(21850,)


In [0]:
#Converting categorical features to One hot encoded features
#clean_categories
vectorizer = LabelEncoderExt()
vectorizer.fit(X_tr['clean_categories'].values)
categories_one_hot_train = vectorizer.transform(X_tr['clean_categories'].values)
categories_one_hot_cv = vectorizer.transform(X_cv['clean_categories'].values)
categories_one_hot_test = vectorizer.transform(X_test['clean_categories'].values)

#clean_subcategories
vectorizer = LabelEncoderExt()
vectorizer.fit(X_tr['clean_subcategories'].values)
subcategories_one_hot_train = vectorizer.transform(X_tr['clean_subcategories'].values)
subcategories_one_hot_cv = vectorizer.transform(X_cv['clean_subcategories'].values)
subcategories_one_hot_test = vectorizer.transform(X_test['clean_subcategories'].values)

#school_state
vectorizer = LabelEncoderExt()
vectorizer.fit(X_tr['school_state'].values)
schoolstate_one_hot_train = vectorizer.transform(X_tr['school_state'].values)
schoolstate_one_hot_cv = vectorizer.transform(X_cv['school_state'].values)
schoolstate_one_hot_test = vectorizer.transform(X_test['school_state'].values)


#project_grade_category
vectorizer = LabelEncoderExt()
vectorizer.fit(X_tr['project_grade_category'].values)
project_grade_category_one_hot_train = vectorizer.transform(X_tr['project_grade_category'].values)
project_grade_category_one_hot_cv = vectorizer.transform(X_cv['project_grade_category'].values)
project_grade_category_one_hot_test = vectorizer.transform(X_test['project_grade_category'].values)



In [0]:
#Concatenating numerical features
rem_input_train = np.concatenate((X_tr['quantity'].values.reshape(-1,1),X_tr['price'].values.reshape(-1,1),X_tr['teacher_number_of_previously_posted_projects'].values.reshape(-1,1)), axis=1)
rem_input_cv = np.concatenate((X_cv['quantity'].values.reshape(-1,1),X_cv['price'].values.reshape(-1,1),X_cv['teacher_number_of_previously_posted_projects'].values.reshape(-1,1)), axis=1)
rem_input_test = np.concatenate((X_test['quantity'].values.reshape(-1,1),X_test['price'].values.reshape(-1,1),X_test['teacher_number_of_previously_posted_projects'].values.reshape(-1,1)), axis=1)

In [16]:
y_train = to_categorical(y_tr)
y_cv = to_categorical(y_cv)
y_test = to_categorical(y_test)
y_test.shape

(21850, 2)

In [0]:
feature_names[1]

In [0]:
feature_names[5]

In [17]:
#Integer encoding Essay column using tokenizer API
#https://machinelearningmastery.com/use-word-embedding-layers-deep-learning-keras/
t = Tokenizer()
t.fit_on_texts(X_tr['essay'])
vocab_size = len(t.word_index) + 1
print("Vocabulary size_train:",vocab_size)
max_length = 400
# integer encode the train data
encoded_docs = t.texts_to_sequences(X_tr['essay'])
essay_pad_train = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
print(essay_pad_train.shape)


# integer encode the cv data
encoded_docs = t.texts_to_sequences(X_cv['essay'])
essay_pad_cv = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
print(essay_pad_cv.shape)


# integer encode the test data
encoded_docs = t.texts_to_sequences(X_test['essay'])
essay_pad_test = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
print(essay_pad_test.shape)



Vocabulary size_train: 47307
(69918, 400)
(17480, 400)
(21850, 400)


In [18]:
#Embedding using Glove vectors
embeddings_index = dict()
f = open(r'/content/drive/My Drive/Applied ML assignments/glove.6B.300d.txt')
#with open('/content/drive/My Drive/Applied ML assignments/glove_vectors', 'rb') as f:
  #text = f.read()
for line in f:
  #line.decode(errors='ignore')
  values = line.split()
  word = values[0]
  coefs = np.asarray(values[1:], dtype='float32')
  embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

Loaded 400000 word vectors.


In [19]:
#create a weight matrix for words in training docs
embedding_matrix = zeros((vocab_size, 300))
for word, i in t.word_index.items():
	embedding_vector = embeddings_index.get(word)
	if embedding_vector is not None:
		embedding_matrix[i] = embedding_vector

print("embedding matrix shape",embedding_matrix.shape)

embedding matrix shape (47307, 300)


In [20]:
#Flattening the text input data after calculating embedding matrix using glove vectors
ins = []
concat = []
text_input = Input(shape=(max_length,), name = "text_input")
# max_length = 400 ---->max length of sentence
ins.append(text_input)
e1 = Embedding(vocab_size, 300, weights=[embedding_matrix], input_length=max_length,trainable=False)(text_input)

l1= LSTM(128,kernel_regularizer=l2(0.001),return_sequences=True)(e1)
#l1= LeakyReLU(alpha = 0.3)(l1)
f1= Flatten()(l1)
concat.append(f1)

In [0]:
#Combining Categorical features
#https://medium.com/@davidheffernan_99410/an-introduction-to-using-categorical-embeddings-ee686ed7e7f9

cat_vars = ["teacher_prefix","school_state","project_grade_category","clean_categories","clean_subcategories"]

cat_sizes = {}
cat_embsizes = {}
for cat in cat_vars:
    cat_sizes[cat] = X_tr[cat].nunique()#nunique - includes unique elements
    cat_embsizes[cat] = min(50, cat_sizes[cat]//2+1)#embedding size is chosen as half the size of unique elements + 1

In [0]:
#Now we iterate over our categorical variables and create an input layer → embedding layer → reshape layer
for cat in cat_vars:
    x = Input((1,), name=cat)
    ins.append(x)
    x = Embedding(cat_sizes[cat]+1, cat_embsizes[cat], input_length=1)(x)
    x = Flatten()(x)
    concat.append(x)

In [0]:
#Converting the remaining input using Dense layer
rem_input_layer =  Input(shape=(3,), name="rem_input_layer")
ins.append(rem_input_layer)
rem_input_dense = Dense(64, activation='relu')(rem_input_layer)
concat.append(rem_input_dense)

In [24]:
#After concatenating text input, categorical and remaining numerical features, applying it to the model
from keras.layers import Concatenate
x = Concatenate()(concat)
#X=BatchNormalization()(X)
x= Dense(256,kernel_initializer='glorot_normal',kernel_regularizer=l2(0.002))(x)
#x= LeakyReLU(alpha = 0.3)(x)
x= Dropout(0.6)(x)
x= Dense(128,kernel_initializer='glorot_normal',kernel_regularizer=l2(0.002))(x)
#x= LeakyReLU(alpha = 0.3)(x)
x= Dropout(0.5)(x)
x= Dense(64,kernel_initializer='glorot_normal',kernel_regularizer=l2(0.002))(x)
#x= LeakyReLU(alpha = 0.3)(x)
x= Dropout(0.5)(x)
x= Dense(32,kernel_initializer='glorot_normal',kernel_regularizer=l2(0.002))(x)
#x= LeakyReLU(alpha = 0.3)(x)
x= Dropout(0.5)(x)
#x=BatchNormalization()(x)
x= Dense(16,activation='relu',kernel_initializer='glorot_normal',kernel_regularizer=l2(0.002))(x)
#x= LeakyReLU(alpha = 0.3)(x)
#x= Dropout(0.25)(x)
output=Dense(2, activation='softmax')(x)
model_l = Model(inputs=ins, outputs=output)
model_l.summary()
#,kernel_initializer='glorot_normal',kernel_regularizer=l2(0.002)



Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text_input (InputLayer)         (None, 400)          0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 400, 300)     14192100    text_input[0][0]                 
__________________________________________________________________________________________________
teacher_prefix (InputLayer)     (None, 1)            0                                            
__________________________________________________________________________________________________
school_state (InputLayer)       (None, 1)            0                    

In [0]:
ins.

In [0]:
import tensorflow as tf
from sklearn.metrics import roc_auc_score

def auroc(y_true, y_pred):
    return tf.py_func(roc_auc_score, (y_true, y_pred), tf.double)


In [26]:
import keras
adam = keras.optimizers.Adam(lr=0.001)
model_l.compile(optimizer=adam, loss='categorical_crossentropy',metrics=[auroc])



Instructions for updating:
tf.py_func is deprecated in TF V2. Instead, there are two
    options available in V2.
    - tf.py_function takes a python function which manipulates tf eager
    tensors instead of numpy arrays. It's easy to convert a tf eager tensor to
    an ndarray (just call tensor.numpy()) but having access to eager tensors
    means `tf.py_function`s can use accelerators such as GPUs as well as
    being differentiable using a gradient tape.
    - tf.numpy_function maintains the semantics of the deprecated tf.py_func
    (it is not differentiable, and manipulates numpy arrays). It drops the
    stateful argument making all functions stateful.
    


In [0]:
from keras.callbacks import *
es = EarlyStopping(monitor='val_loss', mode='min', patience=10,verbose=1)

batch_size = 512
filepath = '/content/drive/My Drive/Applied ML assignments/Epoch/epochs:{epoch:03d}-val_auc:{val_auroc:.3f}.hdf5'
#earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save = ModelCheckpoint(filepath, save_best_only=True, monitor='val_auc', mode='max')
reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=1, verbose=1,min_lr=0.001, mode='min')
callbacks=[es, mcp_save, reduce_lr_loss]
#model.fit(Xtr_more, Ytr_more, batch_size=batch_size, epochs=50, verbose=0, callbacks=[earlyStopping, mcp_save, reduce_lr_loss], validation_split=0.25)

In [28]:
history_1= model_l.fit({'text_input': essay_pad_train, 'school_state': schoolstate_one_hot_train, 'project_grade_category': project_grade_category_one_hot_train,'clean_categories': categories_one_hot_train,'clean_subcategories':subcategories_one_hot_train, 'teacher_prefix':teacherprefix_ohe_train, 'rem_input_layer':rem_input_train},
          y_train,epochs=20, batch_size=512,verbose=1, 
          validation_data=({'text_input': essay_pad_cv, 'school_state': schoolstate_one_hot_cv, 'project_grade_category': project_grade_category_one_hot_cv,'clean_categories': categories_one_hot_cv,'clean_subcategories':subcategories_one_hot_cv, 'teacher_prefix':teacherprefix_ohe_cv, 'rem_input_layer':rem_input_cv},
          y_cv),callbacks=callbacks)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 69918 samples, validate on 17480 samples
Epoch 1/20
69918/69918 [==============================] - 1093s 16ms/step - loss: 3.6809 - auroc: 0.4961 - val_loss: 3.1844 - val_auroc: 0.5000
Epoch 2/20
69918/69918 [==============================] - 1079s 15ms/step - loss: 3.0693 - auroc: 0.4969 - val_loss: 2.9966 - val_auroc: 0.4979
Epoch 3/20
69918/69918 [==============================] - 1081s 15ms/step - loss: 2.9487 - auroc: 0.4902 - val_loss: 2.9057 - val_auroc: 0.4895
Epoch 4/20
69918/69918 [==============================] - 1104s 16ms/step - loss: 2.8859 - auroc: 0.4809 - val_loss: 2.8666 - val_auroc: 0.4806
Epoch 5/20
69918/69918 [==============================] - 1080s 15ms/step - loss: 2.5507 - auroc: 0.4643 - val_loss: 0.9984 - val_auroc: 0.4263
Epoch 6/20
69918/69918 [==============================] - 1076s 15ms/step - loss: 0.8215 - auroc: 0.4935 - val_loss: 0.7143 - val_auro

In [29]:
custom_objects = {"auroc":auroc}
#from keras.models import load_model
#best_model_2 = load_model('/content/drive/My Drive/Applied ML assignments/Epoch/epochs:011-val_acc:0.673.hdf5',custom_objects=custom_objects)
result = model_l.evaluate({'text_input': essay_pad_test, 'school_state': schoolstate_one_hot_test, 'project_grade_category': project_grade_category_one_hot_test,'clean_categories': categories_one_hot_test,'clean_subcategories':subcategories_one_hot_test, 'teacher_prefix':teacherprefix_ohe_test, 'rem_input_layer':rem_input_test},
          y_test,batch_size=512)

21850/21850 [==============================] - 119s 5ms/step


In [30]:
print("{} of test data {}". format(model_l.metrics_names[0],result[0]))
print("{} of test data {}". format(model_l.metrics_names[1],result[1]))

loss of test data 0.4113458995306246
auroc of test data 0.7382636455252072
